# Neo4j and the Offshore Leaks: the Case of Azerbaijan

Original article: https://neo4j.com/graphgists/neo4j-and-the-offshore-leaks-the-case-of-azerbaijan/

## Objective

We want to explore [how the President of Azerbaijan (for example) is connected to offshore accounts](https://www.icij.org/investigations/offshore/offshore-companies-provide-link-between-corporate-mogul-and-azerbaijans-president/). Why does this matter? Azerbaijani law forbids state officials involved in overseeing business from being involved in business themselves, including being shareholders in companies. In order to understand his dealings, we need to focus on the network he uses to control his assets stored in offshore entities. This network includes family members, companies, addresses, and a complex set of intermediaries and partners.

Create sample dataset:

```sql
// Create People
CREATE (Ilham_Aliyev:Person {first_name:'Ilham', last_name:'Aliyev', job:'President', Employer:'Azerbaijan', picture: 'https://bigbrowser.blog.lemonde.fr/files/2013/04/000_Par7450762-530x343.jpg'})
CREATE (Hassan_Gozal:Person {first_name:'Hassan', last_name:'Gozal', job:'Vice President', Employer:'Intersun Holding', picture: ''})
CREATE (Abdolbari_Gozal:Person {first_name:'Abdolbari', last_name:'Gozal', job:'President', Employer:'Intersun Holding', picture: 'https://www.azersun.az/news/news146.jpg'})
CREATE (Mehriban_Aliyeva:Person {first_name:'Mehriban', last_name:'Aliyeva', job:'Director', Employer:'Heydar Aliyev Foundation ', picture: 'https://www.mehriban-aliyeva.org//u/a/4/o.jpg'})
CREATE (Arzu_Aliyeva:Person {first_name:'Arzu', last_name:'Aliyeva', job:'', Employer:'', picture: 'https://legend.az/uploads/posts/2011-09/1315158828_arzi-aliyeva-008.jpg'})
CREATE (Leyla_Aliyeva:Person {first_name:'Leyla', last_name:'Aliyeva', job:'', Employer:'', picture: 'https://www2.pictures.zimbio.com/gi/Leyla+Aliyeva+Fly+Bakou+Exhibition+Launch+Nn74uXjDl_cl.jpg'})
CREATE (Heydar_Aliyev:Person {first_name:'Heydar', last_name:'Aliyev', job:'', Employer:'', picture: ''})
CREATE (Ridzuan_Salleh:Person {first_name:'Ridzuan', last_name:'Salleh', job:'', Employer:'', picture: ''})

// Create Addresses
CREATE (Al_Fairooz_Dubai_Marina:Address {address:'13 Al Fairooz Dubai Marina', city:'Dubai', country:'Dubai'})
CREATE (Portcullis_TrustNet_Chambers_POBox:Address {address:'Portcullis TrustNet Chambers P.O. Box 3444 Road Town', city:'Tortola', country:'Tortola'})
CREATE (UnitPL01A_Plaza_Level:Address {address:'Unit PL 01-A, Plaza Level, No. 45 Block A Medan Setia 1, Plaza Damansara, Damansara Heights, 50490', city:'Kuala Lumpur', country:'Kuala Lumpur'})
CREATE (Apartment_No1801_Dubai:Address {address:'Apartment No. 1801 Dubai Marina Lerev Residential', city:'Dubai', country:'Dubai'})
CREATE (Apartment_No1802_Dubai:Address {address:'Apartment No. 1802 Dubai Marina Lerev Residential', city:'Dubai', country:'Dubai'})
CREATE (Villa22_Gate51_Avenue6:Address {address:'Villa 22 Gate 51 Avenue 6 Jannusan 504', city:'', country:''})
CREATE (Prospekt_Kutuzonskiy:Address {address:'444, 4/2 Prospekt Kutuzonskiy', city:'Moscow', country:'Moscow'})
CREATE (Honest_Bright_Company_Ltd:Address {address:'Honest & Bright Company Ltd. Office 53-54, Construction 3 3-rd Tverskaya-Yamskaya Street, Bld. 12', city:'Moscow', country:'Moscow'})
CREATE (PO_Box_117920:Address {address:'P.O. Box 117920 Jebel Ali Free Zone', city:'Dubai', country:'Dubai'})
CREATE (Shenton_Way_DBS:Address {address:'6 Shenton Way #14-01 DBS Building Tower One Singapore 068809', city:'Singapore', country:'Singapore'})

// Create Companies
CREATE (Harvard_Management_Limited:Company {name:'Harvard Management Limited', form:'Standard International Company', incorporation:'07/11/2008', status:'Active', link: 'https://offshoreleaks.icij.org/nodes/166436'})
CREATE (LaBelleza_Holdings_Limited:Company {name:'LaBelleza Holdings Limited', form:'Business Company Limited by Shares', incorporation:'07/11/2008', status:'Active', link: 'https://offshoreleaks.icij.org/nodes/166434'})
CREATE (Arbor_Investments_Limited:Company {name:'Arbor Investments Limited', form:'Business Company Limited by Shares', incorporation:'07/11/2008', status:'Active', link: 'https://offshoreleaks.icij.org/nodes/166435'})
CREATE (Portcullis_Trustnet:Company {name:'Portcullis Trustnet', form:'OFFSHORE SERVICE PROVIDER', incorporation:'', status:'', link: 'https://offshoreleaks.icij.org/nodes/54662'})
CREATE (Naziq_and_Partners:Company {name:'Naziq & Partners', form:'', incorporation:'', status:'', link: 'https://offshoreleaks.icij.org/nodes/294050'})
CREATE (Crovelent_Holdings_LTD:Company {name:'Crovelent Holdings LTD.', form:'Business Company Limited by Shares', incorporation:'29/03/2007', status:'Active', link: 'https://offshoreleaks.icij.org/nodes/204584'})
CREATE (Rosamund_International_Ltd:Company {name:'Rosamund International Ltd', form:'Standard International Company', incorporation:'08/11/2002', status:'Inactive', link: 'https://offshoreleaks.icij.org/nodes/138523'})
CREATE (DBS_Trustee_Limited:Company {name:'DBS Trustee Limited', form:'', incorporation:'', status:'', link: 'https://offshoreleaks.icij.org/nodes/290319'})

// Create Relationships
CREATE (Ilham_Aliyev)-[:FAMILY]->(Mehriban_Aliyeva)
CREATE (Ilham_Aliyev)-[:FAMILY]->(Arzu_Aliyeva)
CREATE (Ilham_Aliyev)-[:FAMILY]->(Leyla_Aliyeva)
CREATE (Ilham_Aliyev)-[:FAMILY]->(Heydar_Aliyev)
CREATE (Mehriban_Aliyeva)-[:FAMILY]->(Arzu_Aliyeva)
CREATE (Mehriban_Aliyeva)-[:FAMILY]->(Leyla_Aliyeva)
CREATE (Mehriban_Aliyeva)-[:FAMILY]->(Heydar_Aliyev)
CREATE (Leyla_Aliyeva)-[:FAMILY]->(Arzu_Aliyeva)
CREATE (Leyla_Aliyeva)-[:FAMILY]->(Heydar_Aliyev)
CREATE (Arzu_Aliyeva)-[:FAMILY]->(Heydar_Aliyev)
CREATE (Hassan_Gozal)-[:FAMILY]->(Abdolbari_Gozal)
CREATE (Hassan_Gozal)-[:USES_ADDRESS]->(Al_Fairooz_Dubai_Marina)
CREATE (Harvard_Management_Limited)-[:USES_ADDRESS]->(Portcullis_TrustNet_Chambers_POBox)
CREATE (Naziq_and_Partners)-[:USES_ADDRESS]->(UnitPL01A_Plaza_Level)
CREATE (Arzu_Aliyeva)-[:USES_ADDRESS]->(Apartment_No1801_Dubai)
CREATE (Leyla_Aliyeva)-[:USES_ADDRESS]->(Apartment_No1802_Dubai)
CREATE (Ridzuan_Salleh)-[:USES_ADDRESS]->(Villa22_Gate51_Avenue6)
CREATE (Leyla_Aliyeva)-[:USES_ADDRESS]->(Prospekt_Kutuzonskiy)
CREATE (Mehriban_Aliyeva)-[:USES_ADDRESS]->(PO_Box_117920)
CREATE (Ilham_Aliyev)-[:USES_ADDRESS]->(PO_Box_117920)
CREATE (Rosamund_International_Ltd)-[:USES_ADDRESS]->(Portcullis_TrustNet_Chambers_POBox)
CREATE (DBS_Trustee_Limited)-[:USES_ADDRESS]->(Shenton_Way_DBS)
CREATE (LaBelleza_Holdings_Limited)-[:USES_ADDRESS]->(Portcullis_TrustNet_Chambers_POBox)
CREATE (Arbor_Investments_Limited)-[:USES_ADDRESS]->(Portcullis_TrustNet_Chambers_POBox)
CREATE (Crovelent_Holdings_LTD)-[:USES_ADDRESS]->(Honest_Bright_Company_Ltd)
CREATE (Portcullis_Trustnet)-[:IS_OFFSHORE_PROVIDER_OF]->(Harvard_Management_Limited)
CREATE (Portcullis_Trustnet)-[:IS_OFFSHORE_PROVIDER_OF]->(Naziq_and_Partners)
CREATE (Portcullis_Trustnet)-[:IS_OFFSHORE_PROVIDER_OF]->(LaBelleza_Holdings_Limited)
CREATE (Portcullis_Trustnet)-[:IS_OFFSHORE_PROVIDER_OF]->(Arbor_Investments_Limited)
CREATE (Portcullis_Trustnet)-[:IS_OFFSHORE_PROVIDER_OF]->(Rosamund_International_Ltd)
CREATE (Naziq_and_Partners)-[:IS_LINKED_TO {role:'Master Client', date:''}]->(Harvard_Management_Limited)
CREATE (Portcullis_Trustnet)-[:IS_LINKED_TO {role:'Records And Registers', date:''}]->(Harvard_Management_Limited)
CREATE (Hassan_Gozal)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Harvard_Management_Limited)
CREATE (Ridzuan_Salleh)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Harvard_Management_Limited)
CREATE (Leyla_Aliyeva)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Harvard_Management_Limited)
CREATE (Leyla_Aliyeva)-[:IS_LINKED_TO {role:'Shareholder', date:'39759'}]->(Harvard_Management_Limited)
CREATE (Naziq_and_Partners)-[:IS_LINKED_TO {role:'Master Client', date:''}]->(LaBelleza_Holdings_Limited)
CREATE (Portcullis_Trustnet)-[:IS_LINKED_TO {role:'Records And Registers', date:''}]->(LaBelleza_Holdings_Limited)
CREATE (Hassan_Gozal)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(LaBelleza_Holdings_Limited)
CREATE (Ridzuan_Salleh)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(LaBelleza_Holdings_Limited)
CREATE (Leyla_Aliyeva)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(LaBelleza_Holdings_Limited)
CREATE (Leyla_Aliyeva)-[:IS_LINKED_TO {role:'Shareholder', date:'39759'}]->(LaBelleza_Holdings_Limited)
CREATE (Naziq_and_Partners)-[:IS_LINKED_TO {role:'Master Client', date:''}]->(Arbor_Investments_Limited)
CREATE (Portcullis_Trustnet)-[:IS_LINKED_TO {role:'Records And Registers', date:''}]->(Arbor_Investments_Limited)
CREATE (Arzu_Aliyeva)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Arbor_Investments_Limited)
CREATE (Hassan_Gozal)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Arbor_Investments_Limited)
CREATE (Ridzuan_Salleh)-[:IS_LINKED_TO {role:'Director', date:'39759'}]->(Arbor_Investments_Limited)
CREATE (Arzu_Aliyeva)-[:IS_LINKED_TO {role:'Shareholder', date:'39759'}]->(Arbor_Investments_Limited)
CREATE (Leyla_Aliyeva)-[:IS_LINKED_TO {role:'Shareholder', date:'39653'}]->(Crovelent_Holdings_LTD)
CREATE (Mehriban_Aliyeva)-[:IS_LINKED_TO {role:'Director Of', date:'37635'}]->(Rosamund_International_Ltd)
CREATE (Mehriban_Aliyeva)-[:IS_LINKED_TO {role:'Shareholder Of', date:'37635'}]->(Rosamund_International_Ltd)
CREATE (DBS_Trustee_Limited)-[:IS_LINKED_TO {role:'Master Client', date:''}]->(Rosamund_International_Ltd)
CREATE (Portcullis_Trustnet)-[:IS_LINKED_TO {role:'Records And Registers', date:''}]->(Rosamund_International_Ltd)
CREATE (Ilham_Aliyev)-[:IS_LINKED_TO {role:'Director', date:'37635'}]->(Rosamund_International_Ltd)
CREATE (Ilham_Aliyev)-[:IS_LINKED_TO {role:'Shareholder', date:'37635'}]->(Rosamund_International_Ltd)
```

Sample dataset visualized:

![](img/az004.png)

### President Ilham Aliyev's Direct Assets

We look for the _direct_ links between the President and offshore accounts.

```sql
MATCH (president:Person {first_name:'Ilham'})-[r]->(account:Company) // Find a Person with first name 'Ilham' that is one hop away fom a company
RETURN account.name as Company, account.form as Form, account.incorporation as Incorporation, account.status as Status, r.date as Date, r.role as Role;
```

![](img/az001.png)

### President Ilham Aliyev's Indirect Assets

People who are trying to hide money tend to use proxies they can hide behind.
That means that we must enlarge our search and look for indirect connections.

With a Neo4j database for example, finding all the foreign assets Ilham Aliyev controls directly or indirectly is as simple as adding a `*` to our first query.
The search will return all the paths in the data between Ilham Aliyev and offshore accounts.

```sql
MATCH (president:Person {first_name:'Ilham'})-[r*]->(account:Company)
RETURN DISTINCT account.name as Company, account.form as Form, account.incorporation as Incorporation, account.status as Status

```

![](img/az002.png)

### The role of middlemen

Some middlemen might be particularly well connected and/or important to President Ilham Aliyev.
We can use Cypher find every company in the president's extended network.
We can then find all companies associated with these companies and quantify how tightly connected these middleman companies are to the in-network companies.

```sql
MATCH (president:Person {first_name:'Ilham'})-[r*]->(account:Company)
WITH account
MATCH (account)-[t]-(middlemen:Company)
RETURN middlemen.name as name, count(DISTINCT t) as mentions, type(t) as type, t.role as role
ORDER BY mentions DESC
```

![](img/az003.png)